![uc3m](uc3m.jpg)

# The maximum flow problem

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>

## Summary

Model in Pyomo the maximum flow problem in the Google OR-Tools: https://developers.google.com/optimization/flow/maxflow

![uc3m](maxflow.png)




    



## Formulation with Pyomo



### The data



In [1]:
%%writefile maxflow.dat
set N := node0 node1 node2 node3 node4;
set A := (node0,node1) (node0,node2) (node0,node3) (node1,node2) (node1,node4) (node2,node3) (node2,node4) (node3,node2) (node3,node4);
    
param source := node0;
param sink := node4;
param: capacity :=
node0 node1 20
node0 node2 30
node0 node3 10
node1 node2 40
node1 node4 30
node2 node3 10
node2 node4 20
node3 node2 5
node3 node4 20;

Overwriting maxflow.dat


### The model



In [2]:
%%writefile maxflow.py

from pyomo.environ import *

model = AbstractModel()

## Nodes in the network
model.N = Set() # there are 5 (0,1,2,3,4)
## Network arcs
model.A = Set(within = model.N*model.N)  # there are 9 
    # Set this limit since we cannot have more than one arc between two nodes (at most one arc from node i to node j) (there are no loops)
    # It defines expected properties

## Source node
model.source = Param(within = model.N) # it is node 0 (according to the statement)
## Sink node 
model.sink = Param(within = model.N) # it is node 4 (according to the statement)
## Flow capacity limits
model.capacity = Param(model.A) # we cannot exceed these flow capacity limits

#############################
# 'Define decision variables'
#############################
# The flow over each arc
# Number of decision variables = number of arcs. In our case, we have 9 arcs. This means that we have 9 decision variables (x01,x02,x03,x12,x14,x23,x24,x32,x34)
# model.A contains all arcs in an ordered way
model.flow = Var(model.A) 

#############################
#  'Objective'
#############################
# Maximize the flow into the sink node
def obj(model):
    return sum(model.flow[i,j] for (i,j) in model.A if j == value(model.sink))

#############################
# 'Constraints'
#############################

def upper_limit(model, i, j):
    return model.flow[i,j] <= model.capacity[i,j]

def total_flow_rule(model, k):
    # we define a list with both the inflow and the outflow
    flows = [
        sum(model.flow[i,j] for (i,j) in model.A if j == k), # inflow
        sum(model.flow[i,j] for (i,j) in model.A if i == k), # outflow
        ]

    # we define a list with the boolean values between k (model.N)
    # and the value of both the source and the sink
    checks = [k == value(model.source), k == value(model.sink)]

    # we check if one of them is false
    if any(checks):
        return Constraint.Skip

    # we return the Boolean value comparing
    # the result of the inflow and outflow summations
    return (flows[0] == flows[1])

########## ADDING TO THE MODEL #################

# Add the objective function to the model
model.objective = Objective(rule=obj, sense=maximize)

# Add the constraints to the model
model.upper_limit = Constraint(model.A, rule=upper_limit)
model.total_flow = Constraint(model.N, rule=total_flow_rule)

Overwriting maxflow.py


### Obtain the solution

In [3]:
!pyomo solve --solver=gurobi maxflow.py maxflow.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.10] Creating model
[    0.11] Applying solver
[    0.33] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 60.0
    Solver results file: results.yml
[    0.33] Applying Pyomo postprocessing actions
[    0.33] Pyomo Finished


errorcode: 0
retval:
    instance: <pyomo.core.base.PyomoModel.ConcreteModel object at 0x00000186648C7920>
    local:
        time_initial_import: 0.09903740882873535
        usermodel: <module 'maxflow' from 'd:\\UC3M\\3rd year\\Optimization and Analytics\\Assignments\\Optimization-Assignments\\Min_Max Flow\\Exercise to practice the maximum flow problem-20221102\\maxflow.py'>
    options: <pyomo.common.config.ConfigDict object at 0x0000018667ABAFC0>
    results: {'Problem': [{'Name': 'x10', 'Lower bound': 60.0, 'Upper bound': 60.0, 'Number of objectives': 1, 'Number of constraints': 13, 'Number of variables': 10, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 10, 'Number of nonzeros': 22, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model wa

In [4]:
# Display
# !cat results.yml for Unix-like operating system
# !type for Windows and DOS systems (cmd)
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x10
  Lower bound: 60.0
  Upper bound: 60.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 10
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 10
  Number of nonzeros: 22
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject

In [5]:
import yaml
with open('results.yml') as f:
    doc = yaml.load(f,Loader=yaml.FullLoader)
    l1 = doc["Solution"][1]["Variable"]
    l1 = list(l1.items())
    
# positive flows:    
for i in l1:
    print(i[1]["Value"])


20
30
10
20
10
20
20


### Interpretation

This problem has a feasible solution. The solution found for the maximum flow *(objective function)* is **60**.

The flow amounts across each arc are:

$Arc$ | $Flow$ | $Capacity$ 
 -------|---|----
  $node 0$ --> $node 1$ | 20 | 20
  $node 0$ --> $node 2$ | 30 | 30 
  $node 0$ --> $node 3$ | 10 | 10
  $node 1$ --> $node 2$ | 0  | 40 
  $node 1$ --> $node 4$ | 20 | 30 
  $node 2$ --> $node 3$ | 10 | 10 
  $node 2$ --> $node 4$ | 20 | 20 
  $node 3$ --> $node 2$ | 0  | 5 
  $node 3$ --> $node 4$ | 20 | 20 
  
